# **ATP Match Data Elo Ratings**

In this notebook we construct pre and post-match Elo ratings for both players in each match. These can be used to predict winners. We also will attempt to use these as a predictor in a model of win probability alongside other features. 

Note that predictions solely based on Elo ratings generally perform very well. They can predict in the range of 70% of match winners correctly. Some of my research also indicated that they generally outperform regression-type models. So it may interesting to combine these approaches. 

For a little background on Elo ratings, they were originally developed by [Arpad Elo](https://en.wikipedia.org/wiki/Arpad_Elo) as a way to calcualte relative skill for players two player games like chess. They have been widely applied to other games, including tennis. 

The traditional Elo rating is very simple. Players start with a rating of 1500. When player A and player B are pitted against one antoher, the probability of each player winning is a logistic function of the relative ratings:

$$\begin{aligned}
\mathbb{P}(\text{Player A Wins}) &= \frac{1}{1+10^{(\text{Rating}_B - \text{Rating}_A)/400}} \\
\mathbb{P}(\text{Player B Wins}) &= \frac{1}{1+10^{(\text{Rating}_A - \text{Rating}_B)/400}} = 1-\mathbb{P}(\text{Player A Wins})
\end{aligned}$$

After the match is played, the two players ratings are updated as:

$$\begin{aligned}
\text{Rating}_A &\leftarrow \text{Rating}_A + K(\text{Score}_A - \mathbb{P}(\text{Player A Wins})) \\
\text{Rating}_B &\leftarrow \text{Rating}_B + K(\text{Score}_B - \mathbb{P}(\text{Player B Wins}))
\end{aligned}$$

Where $\text{Score}$ is 1 for the player that won but 0 for the player that lost. When the results of a match are consistent with the relative ratings of the players, the winner's rating is increased but by less than if the winner was not expected to win. Similarly, the loser's rating is decreased but by less than if the loser was not expected to lose. The opposite is true when the result of a match runs against expectations. In this case, the system sees that the ratings might be out of line with skill levels and adjusts more dramatically.  

$K$ is a measure of match importance. We will make $K$ a function of the number of matches played and of the tournament level and round of the tournament. We use the ATP point allocations to account for the latter two. That is, Grand Slams award points in the following manner:
- Winner: 2,000
- Runner-Up: 1,200
- Semi-Finalist: 720
- Quarter-Finalist: 360
- 4th Round: 180
- 3rd Round: 90
- 2nd Round: 45
- 1st Round: 10

Masters 1000 events divide these by half. ATP 500 divide in half agian. and ATP 250 does so again. ATP finals give the winner 1500, the runner up 500, semi-finalists 400, and wins in the round robin 200. Our data breaks our Grand Slams, Masters 1000s, ATP finals, and "other tour-level events". So we'll just treat all "other" events as ATP 500, though we could create a tournament level mapping. We'll divide the points by 1000 to get our multiplier that captures match importance, $K_1 = \text{ATP Points}/1000$. Note that we use the points for the next round. For example, for a semi-final the points we use to determine the importance is 1,200 since this is what the winner of a semi-final would lock in if they win the semi-final. 

We also use a second multiplier for number of matches played: $K_2 = 100/((N+1)^{0.25})$, where $N$ is the number of matches the player has played up to the current match. This allows for players with established ratings to be less effected by each additional match. Then the full match importance will be $K = K_1 \times K_2$.



There are several other things that we might want to adjust for relative to the vanilla Elo rating. This include:
- account for 3 vs 5 sets
- account for surface -- right now we calculate seperate ratings by surface
- reduce ratings when players miss long stretches
- account for how much they won by

For now, we just get the vanilla Elo ratings.

Note that this work draws on a few articles:
- [An Introduction to Tennis Elo](http://www.tennisabstract.com/blog/2019/12/03/an-introduction-to-tennis-elo/)
- [Djokovic And Federer Are Vying To Be The Greatest Of All Time](https://fivethirtyeight.com/features/djokovic-and-federer-are-vying-to-be-the-greatest-of-all-time/)
- [An Introduction to Tennis Modelling](https://www.betfair.com.au/hub/an-introduction-to-tennis-modelling/)
- [Brier score composition – A mini-tutorial](https://timvangelder.com/2015/05/18/brier-score-composition-a-mini-tutorial/)
- [Wikipedia - Elo rating system](https://en.wikipedia.org/wiki/Elo_rating_system#:~:text=The%20Elo%20rating%20system%20is,a%20Hungarian%2DAmerican%20physics%20professor.&text=Two%20players%20with%20equal%20ratings,an%20equal%20number%20of%20wins.)

First, let's mount the google drive, import libraries we need, and connect to our data. 

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter 

In [3]:
import pickle
with open('/content/drive/My Drive/ATP Tennis/Database/ATP_Pickle.pickle','rb') as f:
    atp_pickle = pickle.load(f)

To calculate Elo ratings, we first need the matches to be sorted in order of date played. The data we have only has one date per tournament, but we also have the round. So we are able to sort by year, tournament date, tournament ID, and round. We will iterate over each match updating the Elo ratings of the two players. 

In [4]:
Matches = atp_pickle[0]
round_order = dict(Counter(Matches["round"]))
round_order["BR"] = 1
round_order["ER"] = 2
round_order["RR"] = 3
round_order["R128"] = 4
round_order["R64"] = 5
round_order["R32"] = 6
round_order["R16"] = 7
round_order["QF"] = 8
round_order["SF"] = 9
round_order["F"] = 10
Matches['round_order'] = Matches['round'].map(round_order)
Matches = Matches.sort_values(by=['year', 'tourney_date','tourney_id','round_order','match_num']).reset_index(drop=True)
Matches["MatchEloID"] = Matches.index 

Then we assign the match importance for each match based on tournament level and round. 

In [5]:
round_points = round_order
round_points["BR"] = 0.200
round_points["ER"] = 0.200
round_points["RR"] = 0.200
round_points["R128"] = 0.045
round_points["R64"] = 0.090
round_points["R32"] = 0.180
round_points["R16"] = 0.360
round_points["QF"] = 0.720
round_points["SF"] = 1.200
round_points["F"] = 2.000
Matches['match_points'] = Matches['round'].map(round_points)
Matches['match_points'] = np.where(Matches["tourney_level"]=="M",
                                   Matches['match_points']/2,Matches['match_points'])
Matches['match_points'] = np.where(Matches["tourney_level"]=="A",
                                   Matches['match_points']/4,Matches['match_points'])
Matches['match_points'] = np.where((Matches["tourney_level"]=="F") & (Matches['round'] == "F"),
                                   1.500,Matches['match_points'])
Matches['match_points'] = np.where((Matches["tourney_level"]=="F") & (Matches['round'] == "SF"),
                                   0.500,Matches['match_points'])
Matches['match_points'] = np.where((Matches["tourney_level"]=="F") & (Matches['round'] == "QF"),
                                   0.400,Matches['match_points'])

We also need a dictionary of the players' current Elo ratings. All of these start at 1500. They will be updated as we work through each match. We also create a dictionary that will track the current number of matches that each player has played.  We create these and new fields in the match data in a function so we'll be able to create Elo ratings easily fro each surface. 

In [6]:
Players = atp_pickle[1]
Players["Elo_Rating"] = 1500.0
Players["MatchCount"] = 0

def resetElo(Surface):
  EloRatings = dict(zip(Players.player,Players.Elo_Rating))
  MatchCounts = dict(zip(Players.player,Players.MatchCount))
  Matches["PreMatchElo_Winner_"+Surface] = 0
  Matches["PreMatchElo_Loser_"+Surface] = 0
  Matches["PostMatchElo_Winner_"+Surface] = 0
  Matches["PostMatchElo_Loser_"+Surface] = 0
  Matches["PreMatchCount_Winner_"+Surface] = 0
  Matches["PreMatchCount_Loser_"+Surface] = 0
  return EloRatings,MatchCounts

Now we define a function that updates the Elo ratings in our dictionaries and populates the ratings relevant to each match. 

In [7]:
def EloUpdate(MatchID,Surface,Surface_List): 
  '''
  This function updates the Elo ratings and match counts of the players 
  participating in match number MatchID. 
  '''
  #Pull Player IDs, Matches Played, and Pre-Match Elo Ratings
  WinnerID,LoserID = Matches.iloc[MatchID]["winner_id"],Matches.iloc[MatchID]["loser_id"]
  MatchesPlayedWinner,MatchesPlayedLoser = MatchCounts[WinnerID],MatchCounts[LoserID]
  WinnerRating,LoserRating = EloRatings[WinnerID],EloRatings[LoserID]

  #Populate Pre-Match Elo Ratings and Match Counts in Match Data
  Matches.iloc[MatchID,Matches.columns.get_loc("PreMatchElo_Winner_"+Surface)] = WinnerRating
  Matches.iloc[MatchID,Matches.columns.get_loc("PreMatchElo_Loser_"+Surface)] = LoserRating
  Matches.iloc[MatchID,Matches.columns.get_loc("PreMatchCount_Winner_"+Surface)] = MatchesPlayedWinner
  Matches.iloc[MatchID,Matches.columns.get_loc("PreMatchCount_Loser_"+Surface)] = MatchesPlayedLoser

  if Matches.iloc[MatchID]["surface"] in Surface_List:

    #Calculate Player Win Probabilities 
    WinProbA = 1.0 / (1 + 10**((LoserRating - WinnerRating) / 400))
    WinProbB = 1 - WinProbA

    #Match Importance 
    MIATPPoints = Matches.iloc[MatchID]["match_points"]
    MIMatchesPlayedWinner = 100/((MatchesPlayedWinner+1)**0.25)
    MIMatchesPlayedLoser = 100/((MatchesPlayedLoser+1)**0.25)

    #Update Player Ratings based on Current Match Result
    WinnerRating += (1+MIATPPoints)*MIMatchesPlayedWinner*(1-WinProbA)
    LoserRating += (1+MIATPPoints)*MIMatchesPlayedLoser*(-WinProbB) 

    #Update Ratings and Match counts in Dictionaries
    EloRatings[WinnerID],EloRatings[LoserID] = round(WinnerRating,6),round(LoserRating,6)
    MatchCounts[WinnerID] += 1
    MatchCounts[LoserID] += 1

  #Populate Post-Match Elo Ratings in Match Data
  Matches.iloc[MatchID,Matches.columns.get_loc("PostMatchElo_Winner_"+Surface)] = round(WinnerRating,6)
  Matches.iloc[MatchID,Matches.columns.get_loc("PostMatchElo_Loser_"+Surface)] = round(LoserRating,6)


We then run this funciton for each match for all surfaces and seperately for each surface. This is a bit computationally intensive but Elo ratings are very useful, so it's worth it. 

In [8]:
EloRatings,MatchCounts = resetElo("All")
for i in range(Matches.shape[0]):
  EloUpdate(i,"All",["Hard", "Clay","Grass","Carpet",None,'None'])

In [14]:
EloRatings,MatchCounts = resetElo("Grass")
for i in range(Matches.shape[0]):
  EloUpdate(i,"Grass",["Grass"])

In [15]:
EloRatings,MatchCounts = resetElo("Clay")
for i in range(Matches.shape[0]):
  EloUpdate(i,"Clay",["Clay"])

In [16]:
EloRatings,MatchCounts = resetElo("Hard")
for i in range(Matches.shape[0]):
  EloUpdate(i,"Hard",["Hard"])

In [17]:
atp_pickle = [Matches,atp_pickle[1],atp_pickle[2]]
with open('/content/drive/My Drive/ATP Tennis/Database/ATP_Pickle.pickle', 'wb') as f:
    pickle.dump(atp_pickle, f)

We'll dive into what these Elo ratings look like over time in some future notebooks.